# Design of a pseudo-spectral Poisson solver

# A. Motivation

The evolution of a plasma system is tied to field equations. For the case of electrostatics ($B = 0$), we consider Boltzmann-Gauss/Poisson systems. At this juncture, DECSKS apprehends the Vlasov-Poisson system which requires solution to Poisson's equation (or, Gauss' law) in order to evolve this collisionless plasma system self-consistently, according to either equivalent set of equations:

\begin{eqnarray*}
\frac{\partial f}{\partial t} + v\frac{\partial f}{\partial x} + \frac{\partial\phi}{\partial x}\frac{\partial f}{\partial v} = 0, \qquad &  \text{and } & \qquad \frac{\partial^2 \phi}{\partial x^2} = \int_v dv f(t,x,v) - n_0  \\[2em]
& \text{or } & \\[2em]
\frac{\partial f}{\partial t} + v\frac{\partial f}{\partial x} - E\frac{\partial f}{\partial v} = 0, \qquad & \text{and } & \qquad \frac{\partial E}{\partial x} = n_0 -  \int_v dv f(t,x,v)
\end{eqnarray*}

For some domains $x\in \mathcal{D}_x, v\in \mathcal{D}_v$. 

For periodic domains (periodic extensions of non-periodic domains), solving Poisson's equation (Gauss' law) can be facilitated by Fourier-based methods, which inherit the superior convergence and accuracy of spectral methods (also, their hangups, e.g. Gibbs oscillations near sharp transition regions).

# B. Goals of this notebook

This notebook focuses on the design of pseudo-spectral (Fourier-based) solutions of the equations on the right-hand side of the above set. Fourier-based methods operate on globally interpolating functions, giving rise to its well-known convergence and accuracy properties. In [1], a Gauss' law Fourier based method is discussed, we repeat equivalent information (the only difference should be that the reference chooses an asymmetric wave number interval).

Solution of the differential equations proceed by electing to differentiate not the function itself (e.g. finite differencing), but instead to differentiate the unique trigonometric interpolation of the function (i.e. the function is projected onto the subspace spanned by complex-valued exponentials in $\mathbb{C}^{n}$, a <i>Fourier transform</i>) so that differentiation is reduced to algebraic multiplication whereafter the obtained derivative can be mapped back onto $\mathbb{R}^n$ (<i>inverse Fourier transform</i>). The efficiency of the transform procedure can be reduced to $O(N_x\log N_x)$ operations for a mesh characterized by $N_x$ grid points, by a divide-and-conquer strategy that recursively groups wave numbers into bins of progressively half the number of wave numbers $N_x$, which constitute the well-known class of algorithms known as the <i>fast Fourier transform (FFT)</i> (e.g. radix-2). Because of the Fourier basis whose basis functions globally interpolate the function, we inherit spectral-like properties and accuracy that is also well-known to be unmatched by any other means.

# C. Setup and definitions

We consider the forms given in section A [1]:

\begin{eqnarray*}
 \frac{\partial E}{\partial x} = n_0 -  \int_v dv f(t,x,v)\qquad &  \text{or } & \qquad \frac{\partial^2 \phi}{\partial x^2} = \int_v dv f(t,x,v) - n_0 
\end{eqnarray*}

where $\partial_x\phi = -E$, and $\phi = \phi (t,x)$ and $E = E(t,x)$.


The ion density is taken to be a cold background (constant) value, chosen so that $n_i \equiv n_0$ maintains quasineutrality globally, i.e. equal to the total number of charge carriers $N_e$ over the length of the domain $L = b - a$ that constitute the electron density $n_e \equiv n$. On scales of the Debye length (note that in the Vlasov model above, $x$ has been normalized by $\lambda_D$ so that it measures multiples of the Debye length) localized electric fields are not sufficiently screened out, as computed by Poisson's equation/Gauss' law. To find the value of the ion background density, we calculate the total number of electrons $N_e$ for the initial distribution:

$$N_e = \int_a^b dx \int_v dv f(t,x,v) = \int_a^b dx \int_v dv f_0(x,v)$$

then, since the number of positive ions, $\int_a^b dx n_0$, must equal this number, and $n_0 = \text{const}$, we have

$$n_0 = \frac{N_e}{b - a}$$

## C.1. Transform definitions

The projection onto the subspace of complex exponentials permits expressing the function as a trigonometric series:

$$f(x) = \sum_{k = -\infty}^{\infty} \mathcal{F}[f](k) e^{\text{i}\xi_k x}$$

Note, the imaginary unit $\text{i} = \sqrt{-1}$ is formatted in plain text to avoid any confusion with the usual grid index associated with the first configurational variable $i$ that is most often used by this author. In the limit above where an infinite number of waves are added, the RMS error between a Fourier series and the function is zero. The wave number has the form $\xi_k = 2\pi k / L$ for a domain of wave period $T_k$ (related by $k/L = 1 / T_k$, i.e. $\xi_k = 2\pi / T_k$), where $L = b - a$, where $a,b\in \mathbb{R}$. The coefficients are the aforementioned projections which are obvious when viewing the above series.

\begin{eqnarray*}
f(x) & = & \sum_{m  =  -\infty}^{\infty} \mathcal{F}[f](k) e^{\frac{2\pi\text{i}m}{L}}  \\[1em]
\int_{-L/2}^{L/2} dx f(x) e^{\frac{2\pi\text{i}k}{L}}  & = & \int_{-L/2}^{L/2} dx\sum_{m  =  -\infty}^{\infty} \mathcal{F}[f](k) e^{\frac{2\pi\text{i}k}{L}} e^{\frac{2\pi\text{i}m}{L}} \\[1em]
\int_{-L/2}^{L/2} dx f(x) e^{\frac{2\pi\text{i}k}{L}}  & = & \sum_{m  =  -\infty}^{\infty} \mathcal{F}[f](k) \left(\int_{-L/2}^{L/2} dx e^{\frac{2\pi\text{i}(m + k)}{L}}\right) \\[1em]
\int_{-L/2}^{L/2} dx f(x) e^{\frac{2\pi\text{i}k}{L}}  & = & \sum_{m  =  -\infty}^{\infty} \mathcal{F}[f](k)(L\delta_{mk}) \\[1em]
\int_{-L/2}^{L/2} dx f(x) e^{\frac{2\pi\text{i}k}{L}}  & = & \mathcal{F}[f](k) L
\end{eqnarray*}

Thus,

$$\mathcal{F}[f](\xi_k) = \frac{1}{L}\int_{-L/2}^{L/2} f(x) e^{-\frac{2\pi\text{i}}{L}kx}dx = \frac{(f, e^{\frac{2\pi\text{i}}{L}kx})}{(e^{\frac{2\pi\text{i}}{L}kx}, e^{\frac{2\pi\text{i}}{L}kx})} \qquad \underline{\text{Forward (Fourier) transform}}$$

Where the inner product is defined as $(A, B) =  \langle B | A \rangle = \int_{\mathbb{x\in \mathcal{D}}} dx A\bar{B} = \int_{x\in\mathcal{D}} dx B^*A$ (the author favors the mathematicians' notation using paranthesis over the bra-ket notation of physicists, but both have been provided so there is no lack of clarity here). The backward transform is easily understood as:

$$\mathcal{F}^{-1}[f](\xi_k) = f(x) = \int_{-L/2}^{L/2} \mathcal{F}[f](\xi_k) e^{\frac{2\pi\text{i}}{L}kx}dx = (e^{\frac{2\pi\text{i}}{L}kx},f) \qquad \underline{\text{Backward (inverse Fourier) transform}}$$

On a computer, we write $x_m = m\Delta x = m \frac{L}{N_x}$, $m = 0, 1, \ldots , N_x - 1$ and define discrete Fourier transform (DFT) and inverse DFT (IDFT) of the above:

$$\mathcal{F}[f](\xi_k) \simeq \text{DFT}[f]_k  := \frac{1}{N_x} \sum_{m = 0}^{N_x - 1} f(x_m) e^{-\text{i}\xi_k x_m}$$


$$f(x_m) \simeq \text{IDFT}[\text{DFT}[f]]_m  := \sum_{k = 0}^{N_x - 1}\text{DFT}[f]_k e^{\text{i}\xi_k x_m}$$

If a fast Fourier transform (FFT) algorithm is used, then the above transform pair can be written as:

$$\mathcal{F}[f](\xi_k) \simeq \text{FFT}[f]_k  := \frac{1}{N_x} \sum_{m = 0}^{N_x - 1} f(x_m) e^{-\text{i}\xi_k x_m}$$


$$f(x_m) \simeq \text{IFFT}[\text{FFT}[f]]_m  := \sum_{k = 0}^{N_x - 1}\text{FFT}[f]_k e^{\text{i}\xi_k x_m}$$


# D. Gauss' law solver

We solve the discrete problem $\left(\frac{\partial E}{\partial x}\right)_{x = x_i} = n_0 -  \int_v dv f(t,x_i,v)$ at each $x_i$ inside the domain $x\in [a, b]$, and enumerate according to $i = 0, 1, 2, \ldots , N_x - 1$.

The right-hand side can be computed as $\rho_i = n_0 - \sum_j \Delta v_j f_{i,j}^n$ where $f_{i,j}^n \equiv (f(t^n, x_i, v_j)$, and $\rho_i \equiv \rho (x_i)$, then the governing equation s:

$$\left(\frac{\partial E}{\partial x}\right)_{x = x_i} = \rho_i$$

We develop the continuous version, then assert the discrete case holds:

\begin{eqnarray*}
\frac{\partial E}{\partial x} & = & \rho \\
\end{eqnarray*}

Apply the continuous forward transform:

\begin{eqnarray*}
\mathcal{F}\frac{\partial E}{\partial x} & = & \mathcal{F}\rho \\
\frac{1}{L}\int_{-L/2}^{L/2}\frac{\partial E}{\partial x} e^{-\frac{\text{i}\xi_k}{L}x}dx & = & \mathcal{F}\rho \\
\underbrace{\frac{1}{L}\left(Ee^{-\text{i}\xi_k x}\right)_{-L/2}^{L/2}}_{=\, 0,\, \text{periodicity}} - (-\text{i}\xi_k) \underbrace{\frac{1}{L}\int_{-L/2}^{L/2}E e^{-\text{i}\xi_k x}dx}_{= \mathcal{F}E} & = & \mathcal{F}\rho \\
\text{i}\xi_k \mathcal{F}E & = & \mathcal{F}\rho 
\end{eqnarray*}

or,

$$\boxed{\mathcal{F}E = \frac{\mathcal{F}\rho}{\text{i}\xi_k}}$$

so, that the electric field can be computed from Gauss' law swiftly by an inverse of the above quantity:

$$E(t,x) =  \mathcal{F}^{-1}\left[\frac{\mathcal{F}\rho}{\text{i}\xi_k}\right]$$

For the discrete case (using FFT/IFFT), we set up a vectors $\underline{x} = \{x_i\}$, for $i = 0, 1, \ldots , N_x - 1$, $x_i\in [a, b]$, and $\underline{\rho} = \{\rho_i\}$ at those spatial locations. Then the discrete Fourier transform compues a <i>vector</i> as $\text{FFT}[\underline{\rho} \equiv \hat{\underline{\rho}} = \{\hat{\rho}_k\}$, where each element is labeled by the wave index $k$. For transparency, we might write this as $\text{FFT}[\rho]_k - \hat{\rho}_k$ as the $k$th element in the Fourier transform vector of $\underline{\rho}$:

$$\boxed{E(t^n,\underline{x}) = \text{IFFT} \left\{ \left(\frac{\text{FFT}[\rho ]_k}{\text{i}\xi_k}\right)_{k = 0}^{N_x - 1}\right\} } \qquad \qquad (1)$$

Note, the meaning of the inner object above is the scaled transform vector:

$$ \left(\frac{\text{FFT}[\rho ]_k}{\text{i}\xi_k}\right)_{k = 0}^{N_x - 1} = \left(\frac{\rho_0}{\text{i}\xi_0}, \frac{\rho_1}{\text{i}\xi_1}, \frac{\rho_2}{\text{i}\xi_2} , \ldots , \frac{\rho_{N_x-1}}{\text{i}\xi_{N_x -1}} \right), \qquad (2)$$

<b>Caution</b>: in the Fourier calculation of integration, we have shown the following mapping: $\int \mapsto 1 / (\text{i}\xi_k)$ holds. It is emphasized that the $k$th wave number in $1 / (\text{i}\xi_k)$ must be multiplied by the $k$th component of $\text{FFT}[\rho]_k$. That is, the wave numbers will need to be computed (see section D) and multiplied by the element of the Fourier transform vector $\text{FFT}[\rho]_{\ell}$ that actually pertains to the $k$th frequency component. It should <i>not</i> be assumed that the $\ell$th component of $\text{FFT}[\underline{\rho}]$ is the $k$th freqency component...it depends on how the transform vector is organized! Different programming languages are equipped with modules that may use different ordering of the Fourier transformed vector, thus it is necessary to learn what organization is used so that the multiplication as in the above does pertain to integration and is not a random product that does not have any meaning (i.e. it is meainingless to compute $\text{FFT}[\rho ]_{\ell} / (\text{i}\xi_k)$ for $k \neq \ell$). Some languages find it most convenient to put the DC component ($k = 0$) in the center of the actual transform vector, and naturally putting negative frequencies at lower indices (to the left of the center), and positive frequencies to the right (higher indices). Thus, if you are using someone else's FFT routine (as we are, numpy.fft), then it is necessary to find out what kind of result is provided from the function call.  

In the Python numpy.fft module [2]:

"<i>The values in the result follow so-called “standard” order: If A = fft(a, n), then A[0] contains the zero-frequency term (the mean of the signal), which is always purely real for real inputs. Then A[1:n/2] contains the positive-frequency terms, and A[n/2+1:] contains the negative-frequency terms, in order of decreasingly negative frequency. For an even number of input points, A[n/2] represents both positive and negative Nyquist frequency, and is also purely real for real input. For an odd number of input points, A[(n-1)/2] contains the largest positive frequency, while A[(n+1)/2] contains the largest negative frequency</i>"

Thus, in section D, below, we compute a wave number vector $\underline{\xi} = \{\xi_k\}$ for each $k$, where for $k = 1$ to $k = N_x / 2$ we compute $\xi_k = 2\pi k / N_x$, and after the Nyquist frequency at the index $k = N_x / 2$, we compute (as described above in the paste from the numpy.fft documentation) the negative frequency contributions $\xi_k = 2\pi (k - N_x)/ L$ organized in order of decreasingly negative frequency. This sets up the vector $\underline{\xi}$ in the same ordering as transform vectors, say $\text{FFT}[\underline{\rho}]$. Thus, this setup makes the $\ell$th entry in $\text{FFT}[\underline{\rho}]$ agree with the $k$th wave numbers, i.e. $\ell = k$. This nuance is something that needs to be addressed as it perhaps is the most common way to implement Fourier differentiation/integration incorrectly.

## D.1. Algorithm


<ol>
<li> Compute the wave numbers $\xi_k$ so that the $k$th wave number corresponds to the $k$th entry in the transform vectors (steps 2):

$$ \xi_k =
  \begin{cases}
   2\pi k / L & \text{if } k \leq N_x/2 \\[0.5em]
   2\pi (k - N_x) / L       & \text{else}
  \end{cases}$$

Note, this computation recognizes that second half of the wave numbers are equal to the negative wave numbers.

<li> Calculate $\text{FFT}[\underline{\rho}] =  \rho_i = n_0 - \sum_j \Delta v_j f_{i,j}^n$, e.g. using numpy.fft.fft.

<li> Integrate in Fourier space, by performing the the multipication in eq. (2):

$$\left(\frac{\text{FFT}[\rho ]_k}{\text{i}\xi_k}\right)_{k = 0}^{N_x - 1} = \left(\frac{\rho_0}{\text{i}\xi_0}, \frac{\rho_1}{\text{i}\xi_1}, \frac{\rho_2}{\text{i}\xi_2} , \ldots , \frac{\rho_{N_x-1}}{\text{i}\xi_{N_x -1}} \right)$$


Note, in NumPy arrays, a vectorized computation of this does not require looping and can be done by numpy division of two arrays.

<li> Find the electric field by taking the IFFT of the scaled Fourier transform vector in step (3),  per eq. (1)

$$E(t^n,\underline{x}) = \text{IFFT} \left\{ \left(\frac{\text{FFT}[\rho ]_k}{\text{i}\xi_k}\right)_{k = 0}^{N_x - 1}\right\}$$
</ol>

## D.2. Implementation and test case

We consider the solution to Gauss' law:

$$\partial_x E = \rho, \quad x\in [0, 1]$$

where we reuse the chosen solution from DECSKS-04 (6th order FD Poisson solver):

for $\phi (x) = \sin (\pi^2 x)$, this implies $E(x) = -\partial_x\phi (x)$, or

$$E = -\pi^2 \cos(\pi^2 x)$$

It is clear that the 



In [ ]:
import numpy as np

def Gauss(rho, x, v):
    """Computes self-consistent electric field E by solving Gauss' law
    using FFT/IFFT.

    inputs:
    rho -- (ndarray, ndim = 1)  charge density vector,
    f -- (ndarray, dim=2) electron density fe(x,v,n) at time step t^n
                          used to compute ne(x,n) at time step t^n
    x -- (ndarrya, ndim = 1) configurational variable

    outputs:
    E -- (ndarray,dim=1) electric field, E(x) at time t^n
    """
    Nx = len(x)
    L = np.max(x) - np.min(x)
    
    # create wave number vector in same order as Fourier transform vectors
    xi    = np.zeros(Nx)
    for k in range(Nx):
        if k <= Nx/2 :
            xi[k] = 2*np.pi*k / L
        else:
            xi[k] = 2*np.pi*(k - Nx) / L

 
    F_rho = np.fft.fft(rho)
    A    = max(rho)
    eps   = 2.0e-15
    xi_min = A*eps
    for k in range(Nx):
        if np.abs(F_rho[k]) < xi_min:
            N[k] = 0

    # F_E[0] = 0 from periodic BCs, i.e. because N[0] = 0, quasineutrality
    # equivalently, E_hat[0] is the DC field, of which there is none in
    # a quasineutral system of charged particles, only flucutations are present
    # E_hat[0] = 0 already from E_hat vector initialization

    
    F_E = np.zeros(Nx, dtype = complex)
    # Electric field in Fourier space
    for k in range(1,Nx):
        F_E[k] = 1 / (1j*xi[k]) * F_rho[k]         
    
    # Electric field in configurational space
    E = np.real(np.fft.ifft(F_E))    

    return E

# E. Poisson's equation solver

We consider the equation:

$$\frac{\partial^2 \phi}{\partial x^2} = \rho$$

Where the right-hand side is casted as a charge density $\rho$ as in section D. From eq. (1), we note that $E = -\partial_x \phi$ indicates

$$\boxed{\partial_x\underline{\phi} = -\text{IFFT} \left\{ \left(\frac{\text{FFT}[\rho ]_k}{\text{i}\xi_k}\right)_{k = 0}^{N_x - 1}\right\} } \qquad \qquad (3)$$

where, again, 

$$ \left(\frac{\text{FFT}[\rho ]_k}{\text{i}\xi_k}\right)_{k = 0}^{N_x - 1} = \left(\frac{\rho_0}{\text{i}\xi_0}, \frac{\rho_1}{\text{i}\xi_1}, \frac{\rho_2}{\text{i}\xi_2} , \ldots , \frac{\rho_{N_x-1}}{\text{i}\xi_{N_x -1}} \right), \qquad (2, \text{revisited})$$

It is clear that integrating once more involves one more division by $\text{i}\xi_k$, so that the following is clear:

$$\boxed{\underline{\phi} = \text{IFFT} \left\{ \left(\frac{\text{FFT}[\rho ]_k}{\xi^2_k}\right)_{k = 0}^{N_x - 1}\right\} } \qquad \qquad (4)$$

for

$$ \left(\frac{\text{FFT}[\rho ]_k}{\xi^2_k}\right)_{k = 0}^{N_x - 1} = \left(\frac{\rho_0}{\xi^2_0}, \frac{\rho_1}{\xi^2_1}, \frac{\rho_2}{\xi^2_2} , \ldots , \frac{\rho_{N_x-1}}{\xi^2_{N_x -1}} \right), \qquad (5)$$

## E.1. Algorithm


<ol>
<li> Compute the wave numbers $\xi_k$ so that the $k$th wave number corresponds to the $k$th entry in the transform vectors (steps 2):

$$ \xi_k =
  \begin{cases}
   2\pi k / L & \text{if } k \leq N_x/2 \\[0.5em]
   2\pi (k - N_x) / L       & \text{else}
  \end{cases}$$

Note, this computation recognizes that second half of the wave numbers are equal to the negative wave numbers.

<li> Calculate $\text{FFT}[\underline{\rho}] =  \rho_i = n_0 - \sum_j \Delta v_j f_{i,j}^n$, e.g. using numpy.fft.fft.

<li> Integrate in Fourier space, 

if $\partial_x\underline{\phi}$ is desired then also compute the multipication in eq. (2):

$$\left(\frac{\text{FFT}[\rho ]_k}{\text{i}\xi_k}\right)_{k = 0}^{N_x - 1} = \left(\frac{\rho_0}{\text{i}\xi_0}, \frac{\rho_1}{\text{i}\xi_1}, \frac{\rho_2}{\text{i}\xi_2} , \ldots , \frac{\rho_{N_x-1}}{\text{i}\xi_{N_x -1}} \right), \qquad (2)$$

otherwise to find $\underline{\phi}$ we require the object in eq. (5):

$$ \left(\frac{\text{FFT}[\rho ]_k}{\xi^2_k}\right)_{k = 0}^{N_x - 1} = \left(\frac{\rho_0}{\xi^2_0}, \frac{\rho_1}{\xi^2_1}, \frac{\rho_2}{\xi^2_2} , \ldots , \frac{\rho_{N_x-1}}{\xi^2_{N_x -1}} \right), \qquad (5)$$
Note, in NumPy arrays, a vectorized computation of this does not require looping and can be done by numpy division of two arrays.

<li> If the derivative $\partial_x\underline{\phi}$ is desired, take the IFFT of eq. (2) per eq. (3):

$$\partial_x\underline{\phi} = -\text{IFFT} \left\{ \left(\frac{\text{FFT}[\rho ]_k}{\text{i}\xi_k}\right)_{k = 0}^{N_x - 1}\right\}$$

If the potential $\underline{\phi}$ is desired, take the IFFT of eq. (5), per eq. (4):

$$\underline{\phi} = \text{IFFT} \left\{ \left(\frac{\text{FFT}[\rho ]_k}{\xi^2_k}\right)_{k = 0}^{N_x - 1}\right\} $$

Note, for a numpy array xi, the computation xi ** 2 computes the vectorized product needed (i.e. in contrast to matrix multiplication, computed in python 2.7+ as xi.dot(xi)).

</ol>

## E.2. Implementation and test case

# References

[1] Y. Güçlü, A.J. Christlieb and W.N.G. Hitchon, Arbitrarily high order Convected Scheme solution of the Vlasov-Poisson system (2013).

[2] Discrete Fourier Transform (numpy.fft):  http://docs.scipy.org/doc/numpy/reference/routines.fft.html#module-numpy.fft